## Data extraction from IMDB

In [1]:
import numpy as np
import pandas as pd
import requests

In [5]:
#!pip install bs4

### Extracting malayalam movies

In [4]:
from bs4 import BeautifulSoup #BeautifulSoup is a function used for webscraping

url = 'https://m.imdb.com/list/ls033322608/'
#above url is the list of movie releases of 2023

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
#soup variable contains the html code of the website
soup

<html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
</body>
</html>

## Extracting titles, summary, cast, poster url from the html code

In [7]:
movie_name = []
overview = []
cast = []
cast_url = []
poster = []
#rest of the things like genre, director, year of release etc are available as csv file

In [8]:
movie_data = soup.findAll('div', attrs = {'class' : 'lister-item mode-detail'})
#taking one part of the html code identified by div tag and class mentioned above
movie_data

[<div class="lister-item mode-detail">
 <div class="lister-item-image ribbonize" data-tconst="tt27431598">
 <a href="/title/tt27431598/"> <img alt="Bramayugam" class="loadlate" data-tconst="tt27431598" height="209" loadlate="https://m.media-amazon.com/images/M/MV5BNWExZDRhZTktMGUxOS00ODQyLTk0ZTMtY2VkYWJkZjZlZDUyXkEyXkFqcGdeQXVyMTQ3Mzk2MDg4._V1_UY209_CR13,0,140,209_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="140"/>
 </a> </div>
 <div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt27431598/">Bramayugam</a>
 <span class="lister-item-year text-muted unbold">(2024)</span>
 </h3>
 <p class="text-muted text-small">
 <span class="certificate">UA</span>
 <span class="ghost">|</span>
 <span class="runtime">139 min</span>
 <span class="ghost">|</span>
 <span class="genre">
 Horror, Thriller            </span>
 </p>
 <div class="ipl-rating-widget">
 <div class="ip

In [9]:
for row in movie_data:
    star = row.find('div', class_ = 'lister-item-content').find_all('p', class_ = 'text-muted text-small')[1].find('span', class_ = "ghost")
    star = star.find_next_siblings()
    break
star
    

[<a href="/name/nm0007123/">Mammootty</a>,
 <a href="/name/nm6976646/">Arjun Ashokan</a>,
 <a href="/name/nm4939794/">Sidharth Bharathan</a>,
 <a href="/name/nm8166766/">Amalda Liz</a>]

In [10]:
for row in movie_data:
    name = row.find('div', class_ = 'lister-item-content').h3.a.text
    movie_name.append(name)
    summary = row.find('div', class_ = 'lister-item-content').find('p', class_ = '').text.replace('\n', '')
    if summary == 'Add a Plot':
        summary = ''
    overview.append(summary)
    actors = []
    actor_url = []
    star = row.find('div', class_ = 'lister-item-content').find_all('p', class_ = 'text-muted text-small')[1].find('span', class_ = "ghost")
    if star:    
        star = star.find_next_siblings()
        for i in star:
            actors.append(i.text)
            actor_url.append(i.get('href'))
    cast.append(actors)
    cast_url.append(actor_url)
    image = row.find('div', class_ = 'lister-item-image ribbonize').a.img.get('loadlate')    
    poster.append(image)


In [11]:
for row in movie_data:
    actors = []
    actor_url = []
    star = row.find('div', class_ = 'lister-item-content').find_all('p', class_ = 'text-muted text-small')[1].find('span', class_ = "ghost")
    if star:    
        star = star.find_next_siblings()
        for i in star:
            actors.append(i.text)
            actor_url.append(i.get('href'))
    break
actors


['Mammootty', 'Arjun Ashokan', 'Sidharth Bharathan', 'Amalda Liz']

In [12]:
print(movie_name[: 5]) #list of titles
print('\n\n')
print(overview[: 5]) #list of overview(summary)
print('\n\n')
print(cast[: 5]) #list of actors
print('\n\n')
print(cast_url[: 5]) #list of actors_url
print('\n\n')
print(poster[: 5]) # list of poster url

['Bramayugam', 'Premalu', 'Aattam', 'Manjummel Boys', 'Malaikottai Vaaliban']



['Thevan, a folk singer of the Paanan caste, has a fateful encounter when escaping slavery, leading to discover an ancient tradition altering his destiny.', 'Sachin pursues romance but finds himself caught between two potential partners, leading to amusing complications.', 'After a party amongst a theatre group, their sole actress, Anjali is subjected to a crime from one of the men in the group. A meeting is called for. In trying to reach a consensus, stories unravel, suspicions surface and clamour ensues.', 'A group of friends get into a daring rescue mission to save their friend from Guna Caves, a perilously deep pit from where nobody has ever been brought back.', 'Journey of Malaikottai Vaaliban, an undisputed warrior, transcends time and geographies, triumphing over every opponent he encounters.']



[['Mammootty', 'Arjun Ashokan', 'Sidharth Bharathan', 'Amalda Liz'], ['Naslen', 'Mamitha Baiju', 'Shyam

## Creating dataframe combining all the lists

In [13]:
print(len(movie_name))
print(len(overview))
print(len(cast))
print(len(poster))

11
11
11
11


In [14]:
movie_name[-1]

'Ram'

In [15]:
movie_data = pd.DataFrame({'Title' : movie_name, 'overview' : overview, 'cast': cast, 'cast_url': cast_url, 'poster': poster})

In [16]:
movie_data.head()

Title                                           overview  \
0            Bramayugam  Thevan, a folk singer of the Paanan caste, has...   
1               Premalu  Sachin pursues romance but finds himself caugh...   
2                Aattam  After a party amongst a theatre group, their s...   
3        Manjummel Boys  A group of friends get into a daring rescue mi...   
4  Malaikottai Vaaliban  Journey of Malaikottai Vaaliban, an undisputed...   

                                                cast  \
0  [Mammootty, Arjun Ashokan, Sidharth Bharathan,...   
1  [Naslen, Mamitha Baiju, Shyam Mohan, Sangeeth ...   
2  [Vinay Forrt, Zarin Shihab, Kalabhavan Shajohn...   
3  [Soubin Shahir, Sreenath Bhasi, Balu Varghese,...   
4  [Mohanlal, Sonalee Kulkarni, Hareesh Peradi, M...   

                                            cast_url  \
0  [/name/nm0007123/, /name/nm6976646/, /name/nm4...   
1  [/name/nm10899733/, /name/nm10310964/, /name/n...   
2  [/name/nm4265894/, /name/nm10487218/, /name/nm...   
3  [/name/nm6229968/, /name/nm5465692/, /name/nm7...   
4  [/name/nm0482320/, /name/nm4619181/, /name/nm5...   

                                              poster  
0  https://m.media-amazon.com/images/M/MV5BNWExZD...  
1  https://m.media-amazon.com/images/M/MV5BYzFmOD...  
2  https://m.media-amazon.com/images/M/MV5BOWU4NT...  
3  https://m.media-amazon.com/images/M/MV5BMTQwZj...  
4  https://m.media-amazon.com/images/M/MV5BZjU1OD...

In [17]:
import os
os.chdir(r"C:\Users\HP\Pictures\DS_Projects\Malayalm_data")
os.getcwd()

'C:\\Users\\HP\\Pictures\\DS_Projects\\Malayalm_data'

In [18]:
#importing the dataset of 2024 movies
m_2024 = pd.read_csv(r"C:\Users\HP\Pictures\DS_Projects\PROJECT\Malayalam_data\BEST MALAYALAM MOVIES 2024.csv")

In [20]:
m_2024#this data is directly downloaded from imdb site

Position       Const     Created    Modified  Description  \
0          1  tt27431598  2024-01-13  2024-01-13          NaN   
1          2  tt28288786  2024-02-22  2024-02-22          NaN   
2          3  tt27302079  2024-01-13  2024-01-13          NaN   
3          4  tt26458038  2024-02-22  2024-02-22          NaN   
4          5  tt23023596  2024-01-13  2024-01-13          NaN   
5          6  tt27816710  2024-01-13  2024-01-13          NaN   
6          7  tt10345546  2024-01-13  2024-01-13          NaN   
7          8  tt29608104  2024-01-13  2024-01-13          NaN   
8          9  tt27470003  2024-01-13  2024-01-13          NaN   
9         10  tt10505918  2024-01-13  2024-01-13          NaN   
10        11  tt11449482  2024-01-13  2024-01-13          NaN   

                   Title                                     URL Title Type  \
0             Bramayugam  https://www.imdb.com/title/tt27431598/      movie   
1                Premalu  https://www.imdb.com/title/tt28288786/      movie   
2                 Aattam  https://www.imdb.com/title/tt27302079/      movie   
3         Manjummel Boys  https://www.imdb.com/title/tt26458038/      movie   
4   Malaikottai Vaaliban  https://www.imdb.com/title/tt23023596/      movie   
5          Abraham Ozler  https://www.imdb.com/title/tt27816710/      movie   
6                 Barroz  https://www.imdb.com/title/tt10345546/      movie   
7                  Turbo  https://www.imdb.com/title/tt29608104/      movie   
8                Bazooka  https://www.imdb.com/title/tt27470003/      movie   
9           L2: Empuraan  https://www.imdb.com/title/tt10505918/      movie   
10                   Ram  https://www.imdb.com/title/tt11449482/      movie   

    IMDb Rating  Runtime (mins)    Year  \
0           8.1           139.0  2024.0   
1           8.3           156.0  2024.0   
2           8.4           139.0  2023.0   
3           8.7           135.0  2024.0   
4           6.3           156.0  2024.0   
5           6.7           143.0  2024.0   
6           NaN             NaN  2024.0   
7           NaN             NaN     NaN   
8           NaN             NaN     NaN   
9           NaN             NaN     NaN   
10          NaN             NaN     NaN   

                                       Genres  Num Votes Release Date  \
0                            Horror, Thriller     5960.0   2024-02-15   
1                             Comedy, Romance     2858.0   2024-02-09   
2                                       Drama     1826.0   2023-10-30   
3                         Adventure, Thriller     5473.0   2024-02-22   
4   Action, Drama, Fantasy, Thriller, Western     3480.0   2024-01-25   
5             Crime, Drama, Mystery, Thriller     1629.0   2024-01-11   
6        Action, Adventure, Fantasy, Thriller        NaN   2024-03-28   
7                                         NaN        NaN          NaN   
8                     Action, Crime, Thriller        NaN          NaN   
9                     Action, Crime, Thriller        NaN          NaN   
10                    Action, Crime, Thriller        NaN          NaN   

               Directors  
0        Rahul Sadasivan  
1            Girish A.D.  
2          Anand Ekarshi  
3            Chidambaram  
4   Lijo Jose Pellissery  
5   Midhun Manuel Thomas  
6               Mohanlal  
7                    NaN  
8           Deeno Dennis  
9   Prithviraj Sukumaran  
10         Jeethu Joseph

In [26]:
m_2024 = m_2024.drop([8, 9])

In [27]:
m_2024

Position       Const     Created    Modified  Description  \
0         1  tt27431598  2024-01-13  2024-01-13          NaN   
1         2  tt28288786  2024-02-22  2024-02-22          NaN   
2         3  tt27302079  2024-01-13  2024-01-13          NaN   
3         4  tt26458038  2024-02-22  2024-02-22          NaN   
4         5  tt23023596  2024-01-13  2024-01-13          NaN   
5         6  tt27816710  2024-01-13  2024-01-13          NaN   

                  Title                                     URL Title Type  \
0            Bramayugam  https://www.imdb.com/title/tt27431598/      movie   
1               Premalu  https://www.imdb.com/title/tt28288786/      movie   
2                Aattam  https://www.imdb.com/title/tt27302079/      movie   
3        Manjummel Boys  https://www.imdb.com/title/tt26458038/      movie   
4  Malaikottai Vaaliban  https://www.imdb.com/title/tt23023596/      movie   
5         Abraham Ozler  https://www.imdb.com/title/tt27816710/      movie   

   IMDb Rating  Runtime (mins)    Year  \
0          8.1           139.0  2024.0   
1          8.3           156.0  2024.0   
2          8.4           139.0  2023.0   
3          8.7           135.0  2024.0   
4          6.3           156.0  2024.0   
5          6.7           143.0  2024.0   

                                      Genres  Num Votes Release Date  \
0                           Horror, Thriller     5960.0   2024-02-15   
1                            Comedy, Romance     2858.0   2024-02-09   
2                                      Drama     1826.0   2023-10-30   
3                        Adventure, Thriller     5473.0   2024-02-22   
4  Action, Drama, Fantasy, Thriller, Western     3480.0   2024-01-25   
5            Crime, Drama, Mystery, Thriller     1629.0   2024-01-11   

              Directors  
0       Rahul Sadasivan  
1           Girish A.D.  
2         Anand Ekarshi  
3           Chidambaram  
4  Lijo Jose Pellissery  
5  Midhun Manuel Thomas

In [28]:
movie_data

Title                                           overview  \
0             Bramayugam  Thevan, a folk singer of the Paanan caste, has...   
1                Premalu  Sachin pursues romance but finds himself caugh...   
2                 Aattam  After a party amongst a theatre group, their s...   
3         Manjummel Boys  A group of friends get into a daring rescue mi...   
4   Malaikottai Vaaliban  Journey of Malaikottai Vaaliban, an undisputed...   
5          Abraham Ozler  Revolves around the personal and professional ...   
6                 Barroz  A 400-year-old spirit, Barroz, assigned to gua...   
7                  Turbo                                                      
8                Bazooka                                                      
9           L2: Empuraan  The journey of Stephen Nedumpilly and how he b...   
10                   Ram  The film focuses on the efforts by the R&AW to...   

                                                 cast  \
0   [Mammootty, Arjun Ashokan, Sidharth Bharathan,...   
1   [Naslen, Mamitha Baiju, Shyam Mohan, Sangeeth ...   
2   [Vinay Forrt, Zarin Shihab, Kalabhavan Shajohn...   
3   [Soubin Shahir, Sreenath Bhasi, Balu Varghese,...   
4   [Mohanlal, Sonalee Kulkarni, Hareesh Peradi, M...   
5    [Jayaram, Jagadish, Senthil Krishna, Arya Salim]   
6   [Mohanlal, Kallirroi Tziafeta, Guru Somasundar...   
7                                                  []   
8   [Mammootty, Gautham Vasudev Menon, Shine Tom C...   
9   [Tovino Thomas, Prithviraj Sukumaran, Mohanlal...   
10  [Mohanlal, Trisha Krishnan, Adil Hussain, Samy...   

                                             cast_url  \
0   [/name/nm0007123/, /name/nm6976646/, /name/nm4...   
1   [/name/nm10899733/, /name/nm10310964/, /name/n...   
2   [/name/nm4265894/, /name/nm10487218/, /name/nm...   
3   [/name/nm6229968/, /name/nm5465692/, /name/nm7...   
4   [/name/nm0482320/, /name/nm4619181/, /name/nm5...   
5   [/name/nm0419688/, /name/nm0415538/, /name/nm3...   
6   [/name/nm0482320/, /name/nm4240443/, /name/nm5...   
7                                                  []   
8   [/name/nm0007123/, /name/nm1069826/, /name/nm3...   
9   [/name/nm5732707/, /name/nm1335387/, /name/nm0...   
10  [/name/nm0482320/, /name/nm1375534/, /name/nm1...   

                                               poster  
0   https://m.media-amazon.com/images/M/MV5BNWExZD...  
1   https://m.media-amazon.com/images/M/MV5BYzFmOD...  
2   https://m.media-amazon.com/images/M/MV5BOWU4NT...  
3   https://m.media-amazon.com/images/M/MV5BMTQwZj...  
4   https://m.media-amazon.com/images/M/MV5BZjU1OD...  
5   https://m.media-amazon.com/images/M/MV5BOWIxYj...  
6   https://m.media-amazon.com/images/M/MV5BYTg1MG...  
7   https://m.media-amazon.com/images/M/MV5BYmQxZD...  
8   https://m.media-amazon.com/images/M/MV5BYWE5MD...  
9   https://m.media-amazon.com/images/M/MV5BZjI1OD...  
10  https://m.media-amazon.com/images/M/MV5BYTU3Mz...

In [29]:
movie_data = movie_data.drop([6,7,8,9,10])

In [30]:
movie_data

Title                                           overview  \
0            Bramayugam  Thevan, a folk singer of the Paanan caste, has...   
1               Premalu  Sachin pursues romance but finds himself caugh...   
2                Aattam  After a party amongst a theatre group, their s...   
3        Manjummel Boys  A group of friends get into a daring rescue mi...   
4  Malaikottai Vaaliban  Journey of Malaikottai Vaaliban, an undisputed...   
5         Abraham Ozler  Revolves around the personal and professional ...   

                                                cast  \
0  [Mammootty, Arjun Ashokan, Sidharth Bharathan,...   
1  [Naslen, Mamitha Baiju, Shyam Mohan, Sangeeth ...   
2  [Vinay Forrt, Zarin Shihab, Kalabhavan Shajohn...   
3  [Soubin Shahir, Sreenath Bhasi, Balu Varghese,...   
4  [Mohanlal, Sonalee Kulkarni, Hareesh Peradi, M...   
5   [Jayaram, Jagadish, Senthil Krishna, Arya Salim]   

                                            cast_url  \
0  [/name/nm0007123/, /name/nm6976646/, /name/nm4...   
1  [/name/nm10899733/, /name/nm10310964/, /name/n...   
2  [/name/nm4265894/, /name/nm10487218/, /name/nm...   
3  [/name/nm6229968/, /name/nm5465692/, /name/nm7...   
4  [/name/nm0482320/, /name/nm4619181/, /name/nm5...   
5  [/name/nm0419688/, /name/nm0415538/, /name/nm3...   

                                              poster  
0  https://m.media-amazon.com/images/M/MV5BNWExZD...  
1  https://m.media-amazon.com/images/M/MV5BYzFmOD...  
2  https://m.media-amazon.com/images/M/MV5BOWU4NT...  
3  https://m.media-amazon.com/images/M/MV5BMTQwZj...  
4  https://m.media-amazon.com/images/M/MV5BZjU1OD...  
5  https://m.media-amazon.com/images/M/MV5BOWIxYj...

In [33]:
m_2024

Position       Const     Created    Modified  Description  \
0         1  tt27431598  2024-01-13  2024-01-13          NaN   
1         2  tt28288786  2024-02-22  2024-02-22          NaN   
2         3  tt27302079  2024-01-13  2024-01-13          NaN   
3         4  tt26458038  2024-02-22  2024-02-22          NaN   
4         5  tt23023596  2024-01-13  2024-01-13          NaN   
5         6  tt27816710  2024-01-13  2024-01-13          NaN   

                  Title                                     URL Title Type  \
0            Bramayugam  https://www.imdb.com/title/tt27431598/      movie   
1               Premalu  https://www.imdb.com/title/tt28288786/      movie   
2                Aattam  https://www.imdb.com/title/tt27302079/      movie   
3        Manjummel Boys  https://www.imdb.com/title/tt26458038/      movie   
4  Malaikottai Vaaliban  https://www.imdb.com/title/tt23023596/      movie   
5         Abraham Ozler  https://www.imdb.com/title/tt27816710/      movie   

   IMDb Rating  Runtime (mins)    Year  \
0          8.1           139.0  2024.0   
1          8.3           156.0  2024.0   
2          8.4           139.0  2023.0   
3          8.7           135.0  2024.0   
4          6.3           156.0  2024.0   
5          6.7           143.0  2024.0   

                                      Genres  Num Votes Release Date  \
0                           Horror, Thriller     5960.0   2024-02-15   
1                            Comedy, Romance     2858.0   2024-02-09   
2                                      Drama     1826.0   2023-10-30   
3                        Adventure, Thriller     5473.0   2024-02-22   
4  Action, Drama, Fantasy, Thriller, Western     3480.0   2024-01-25   
5            Crime, Drama, Mystery, Thriller     1629.0   2024-01-11   

              Directors  
0       Rahul Sadasivan  
1           Girish A.D.  
2         Anand Ekarshi  
3           Chidambaram  
4  Lijo Jose Pellissery  
5  Midhun Manuel Thomas

## Combining the two dataframes

In [34]:
movie_2024 = pd.merge(movie_data, m_2024, on = 'Title')

In [35]:
movie_2024

Title                                           overview  \
0            Bramayugam  Thevan, a folk singer of the Paanan caste, has...   
1               Premalu  Sachin pursues romance but finds himself caugh...   
2                Aattam  After a party amongst a theatre group, their s...   
3        Manjummel Boys  A group of friends get into a daring rescue mi...   
4  Malaikottai Vaaliban  Journey of Malaikottai Vaaliban, an undisputed...   
5         Abraham Ozler  Revolves around the personal and professional ...   

                                                cast  \
0  [Mammootty, Arjun Ashokan, Sidharth Bharathan,...   
1  [Naslen, Mamitha Baiju, Shyam Mohan, Sangeeth ...   
2  [Vinay Forrt, Zarin Shihab, Kalabhavan Shajohn...   
3  [Soubin Shahir, Sreenath Bhasi, Balu Varghese,...   
4  [Mohanlal, Sonalee Kulkarni, Hareesh Peradi, M...   
5   [Jayaram, Jagadish, Senthil Krishna, Arya Salim]   

                                            cast_url  \
0  [/name/nm0007123/, /name/nm6976646/, /name/nm4...   
1  [/name/nm10899733/, /name/nm10310964/, /name/n...   
2  [/name/nm4265894/, /name/nm10487218/, /name/nm...   
3  [/name/nm6229968/, /name/nm5465692/, /name/nm7...   
4  [/name/nm0482320/, /name/nm4619181/, /name/nm5...   
5  [/name/nm0419688/, /name/nm0415538/, /name/nm3...   

                                              poster  Position       Const  \
0  https://m.media-amazon.com/images/M/MV5BNWExZD...         1  tt27431598   
1  https://m.media-amazon.com/images/M/MV5BYzFmOD...         2  tt28288786   
2  https://m.media-amazon.com/images/M/MV5BOWU4NT...         3  tt27302079   
3  https://m.media-amazon.com/images/M/MV5BMTQwZj...         4  tt26458038   
4  https://m.media-amazon.com/images/M/MV5BZjU1OD...         5  tt23023596   
5  https://m.media-amazon.com/images/M/MV5BOWIxYj...         6  tt27816710   

      Created    Modified  Description  \
0  2024-01-13  2024-01-13          NaN   
1  2024-02-22  2024-02-22          NaN   
2  2024-01-13  2024-01-13          NaN   
3  2024-02-22  2024-02-22          NaN   
4  2024-01-13  2024-01-13          NaN   
5  2024-01-13  2024-01-13          NaN   

                                      URL Title Type  IMDb Rating  \
0  https://www.imdb.com/title/tt27431598/      movie          8.1   
1  https://www.imdb.com/title/tt28288786/      movie          8.3   
2  https://www.imdb.com/title/tt27302079/      movie          8.4   
3  https://www.imdb.com/title/tt26458038/      movie          8.7   
4  https://www.imdb.com/title/tt23023596/      movie          6.3   
5  https://www.imdb.com/title/tt27816710/      movie          6.7   

   Runtime (mins)    Year                                     Genres  \
0           139.0  2024.0                           Horror, Thriller   
1           156.0  2024.0                            Comedy, Romance   
2           139.0  2023.0                                      Drama   
3           135.0  2024.0                        Adventure, Thriller   
4           156.0  2024.0  Action, Drama, Fantasy, Thriller, Western   
5           143.0  2024.0            Crime, Drama, Mystery, Thriller   

   Num Votes Release Date             Directors  
0     5960.0   2024-02-15       Rahul Sadasivan  
1     2858.0   2024-02-09           Girish A.D.  
2     1826.0   2023-10-30         Anand Ekarshi  
3     5473.0   2024-02-22           Chidambaram  
4     3480.0   2024-01-25  Lijo Jose Pellissery  
5     1629.0   2024-01-11  Midhun Manuel Thomas

In [1]:
movie_2024.to_csv('1980s__.csv', index = False)

NameError: name 'movie_2023' is not defined

In [36]:
os.getcwd()

'C:\\Users\\HP\\Pictures\\DS_Projects\\Malayalm_data'

In [38]:
os.chdir('C:\\Users\\HP\\Pictures\\DS_Projects\\PROJECT\\Malayalam_data')

In [39]:
data = pd.read_excel("new_data.xlsx")

In [41]:
data.tail()

Title                                    URL  \
1198  Kottaram Veettile Apputtan  https://www.imdb.com/title/tt0273316/   
1199                 Kabooliwala  https://www.imdb.com/title/tt0353594/   
1200                    Vandanam  https://www.imdb.com/title/tt0255668/   
1201                     Priyam   https://www.imdb.com/title/tt0282863/   
1202                      Vettam  https://www.imdb.com/title/tt0400585/   

      IMDb Rating    Year                     Genres  Num Votes Release Date  \
1198          5.7  1998.0     Comedy, Drama, Romance      226.0          NaT   
1199          7.0  1994.0     Comedy, Drama, Romance      296.0          NaT   
1200          7.9  1989.0  Comedy, Romance, Thriller     1300.0          NaT   
1201          5.9  2000.0     Comedy, Drama, Romance      175.0          NaT   
1202          7.1  2004.0            Comedy, Romance     1200.0          NaT   

          Directors                                           overview  \
1198      Rajasenan  A local aristocrat takes a young girl under hi...   
1199  Lal, Siddique  Kannas and Kadalas are rag pickers who enjoy l...   
1200   Priyadarshan  Two cops tries to capture a jail-broken felon,...   
1201       R. Sanal  Though Benny, an orphan, shares a close bond w...   
1202   Priyadarshan  As Gopalakrishnan attempts to smuggle an expen...   

                                                   cast  \
1198                 Jayaram, Shruti, Jagathy Sreekumar   
1199  Vineeth, Jagathy Sreekumar,Innocent Vareed The...   
1200                   Mohanlal, Girija Shettar, Mukesh   
1201              Kunchacko Boban, Deepa Nair, Thilakan   
1202               Dileep, Bhavna Pani, Kalabhavan Mani   

                                                 poster  \
1198                                                NaN   
1199                                                NaN   
1200                                                NaN   
1201                                                NaN   
1202  https://m.media-amazon.com/images/M/MV5BOWNjMj...   

                                             Poster_url  
1198  https://m.media-amazon.com/images/M/MV5BODk2Mj...  
1199  https://m.media-amazon.com/images/M/MV5BYTRlZW...  
1200  https://m.media-amazon.com/images/M/MV5BMjY3MT...  
1201  https://m.media-amazon.com/images/M/MV5BYjkyZD...  
1202  https://m.media-amazon.com/images/M/MV5BOWNjMj...

In [42]:
data1 = pd.concat([data, movie_2024])

In [43]:
data1.shape

(1209, 20)

In [44]:
data.shape

(1203, 12)

In [45]:
data1.drop('poster', axis=1, inplace=True)

In [47]:
data.head()

Title                                    URL  IMDb Rating  \
0  Thoovanathumbikal  https://www.imdb.com/title/tt0249188/          8.6   
1    Thaniyavartanam  https://www.imdb.com/title/tt0237820/          8.8   
2             Innale  https://www.imdb.com/title/tt0248909/          8.0   
3           Yavanika  https://www.imdb.com/title/tt0215376/          7.9   
4     Moonnam Pakkam  https://www.imdb.com/title/tt0249027/          8.3   

     Year          Genres  Num Votes Release Date      Directors  \
0  1987.0  Drama, Romance     4956.0   1987-07-31  P. Padmarajan   
1  1987.0           Drama     3112.0   1987-08-15   Sibi Malayil   
2  1989.0  Drama, Romance      529.0   1989-12-31  P. Padmarajan   
3  1982.0  Drama, Mystery      862.0   1982-04-30    K.G. George   
4  1988.0           Drama     1080.0   1988-08-28  P. Padmarajan   

                                            overview  \
0  Jayakrishnan falls in love with two girls, Rad...   
1  A school teacher living in a village belongs t...   
2  After a serious accident a girl who was hospit...   
3  An infamous tabla player of a touring drama gr...   
4  An old man spends his life in solitude awaitin...   

                                                cast  \
0                Mohanlal Sumalatha Parvathi Ashokan   
1                    Mohanlal Shari Thilakan Vineeth   
2       Mohanlal Thilakan Kaviyoor Ponnamma Parvathi   
3               Shobana Jayaram Srividya Suresh Gopi   
4  Sreenivasan Jayaram Urvashi Innocent Vareed Th...   

                                              poster  \
0  https://m.media-amazon.com/images/M/MV5BNjUyZG...   
1  https://m.media-amazon.com/images/M/MV5BMjBiZj...   
2  https://m.media-amazon.com/images/M/MV5BNDMxZj...   
3  https://m.media-amazon.com/images/M/MV5BM2Y2YW...   
4  https://m.media-amazon.com/images/M/MV5BYzMyNz...   

                                          Poster_url  
0  https://m.media-amazon.com/images/M/MV5BNjUyZG...  
1  https://m.media-amazon.com/images/M/MV5BMjBiZj...  
2  https://m.media-amazon.com/images/M/MV5BNDMxZj...  
3  https://m.media-amazon.com/images/M/MV5BM2Y2YW...  
4  https://m.media-amazon.com/images/M/MV5BYzMyNz...

In [48]:
data1.drop(['cast_url', 'Position', 'Const', 'Created', 'Modified', 'Description', 'Title Type', 'Runtime (mins)'], axis=1, inplace=True)

In [49]:
data1

Title                                     URL  IMDb Rating  \
0      Thoovanathumbikal   https://www.imdb.com/title/tt0249188/          8.6   
1        Thaniyavartanam   https://www.imdb.com/title/tt0237820/          8.8   
2                 Innale   https://www.imdb.com/title/tt0248909/          8.0   
3               Yavanika   https://www.imdb.com/title/tt0215376/          7.9   
4         Moonnam Pakkam   https://www.imdb.com/title/tt0249027/          8.3   
..                   ...                                     ...          ...   
1                Premalu  https://www.imdb.com/title/tt28288786/          8.3   
2                 Aattam  https://www.imdb.com/title/tt27302079/          8.4   
3         Manjummel Boys  https://www.imdb.com/title/tt26458038/          8.7   
4   Malaikottai Vaaliban  https://www.imdb.com/title/tt23023596/          6.3   
5          Abraham Ozler  https://www.imdb.com/title/tt27816710/          6.7   

      Year                                     Genres  Num Votes  \
0   1987.0                             Drama, Romance     4956.0   
1   1987.0                                      Drama     3112.0   
2   1989.0                             Drama, Romance      529.0   
3   1982.0                             Drama, Mystery      862.0   
4   1988.0                                      Drama     1080.0   
..     ...                                        ...        ...   
1   2024.0                            Comedy, Romance     2858.0   
2   2023.0                                      Drama     1826.0   
3   2024.0                        Adventure, Thriller     5473.0   
4   2024.0  Action, Drama, Fantasy, Thriller, Western     3480.0   
5   2024.0            Crime, Drama, Mystery, Thriller     1629.0   

           Release Date             Directors  \
0   1987-07-31 00:00:00         P. Padmarajan   
1   1987-08-15 00:00:00          Sibi Malayil   
2   1989-12-31 00:00:00         P. Padmarajan   
3   1982-04-30 00:00:00           K.G. George   
4   1988-08-28 00:00:00         P. Padmarajan   
..                  ...                   ...   
1            2024-02-09           Girish A.D.   
2            2023-10-30         Anand Ekarshi   
3            2024-02-22           Chidambaram   
4            2024-01-25  Lijo Jose Pellissery   
5            2024-01-11  Midhun Manuel Thomas   

                                             overview  \
0   Jayakrishnan falls in love with two girls, Rad...   
1   A school teacher living in a village belongs t...   
2   After a serious accident a girl who was hospit...   
3   An infamous tabla player of a touring drama gr...   
4   An old man spends his life in solitude awaitin...   
..                                                ...   
1   Sachin pursues romance but finds himself caugh...   
2   After a party amongst a theatre group, their s...   
3   A group of friends get into a daring rescue mi...   
4   Journey of Malaikottai Vaaliban, an undisputed...   
5   Revolves around the personal and professional ...   

                                                 cast  \
0                 Mohanlal Sumalatha Parvathi Ashokan   
1                     Mohanlal Shari Thilakan Vineeth   
2        Mohanlal Thilakan Kaviyoor Ponnamma Parvathi   
3                Shobana Jayaram Srividya Suresh Gopi   
4   Sreenivasan Jayaram Urvashi Innocent Vareed Th...   
..                                                ...   
1   [Naslen, Mamitha Baiju, Shyam Mohan, Sangeeth ...   
2   [Vinay Forrt, Zarin Shihab, Kalabhavan Shajohn...   
3   [Soubin Shahir, Sreenath Bhasi, Balu Varghese,...   
4   [Mohanlal, Sonalee Kulkarni, Hareesh Peradi, M...   
5    [Jayaram, Jagadish, Senthil Krishna, Arya Salim]   

                                           Poster_url  
0   https://m.media-amazon.com/images/M/MV5BNjUyZG...  
1   https://m.media-amazon.com/images/M/MV5BMjBiZj...  
2   https://m.media-amazon.com/images/M/MV5BNDMxZj...  
3   https://m.media-amazon.com/images/M/MV5BM2

In [50]:
data1.to_csv('new_data.csv', index = False)